In [1]:
import numpy as np
import ICESiss as SS
import ICESins as NS
from sklearn import metrics
import processing_tools as pt
import similarity_generator as sg


### Load dataset

In [2]:
p = np.loadtxt("../data/COSN/P.npy")
y = np.loadtxt("../data/COSN/Y.npy")
gt = np.load("../data/COSN/ground_truth.npy")

print(p.shape, y.shape, gt.shape)

(46, 46) (46, 5) (46,)


### preprocessing the data

In [3]:
p_, _, _, p_u, _, _, p_m, _, _, p_l, _, _ = pt.preprocess_p(p=p)


y_, _, y_z, _, y_rng, _, = pt.preprocess_y(y_in=y, data_type='Q')

In [4]:
p_u.shape, y_z.shape

((46, 46), (46, 5))

## Applying ICESi

### ICESiss

In [12]:
np.random.seed(21)  # different seed different clustering results as discuss in the paper
seed = np.random.randint(y_z.shape[0])
seed

9

### Converting feature data to similarity data

- s_inn >> similarity obtained by inner product (which is used in the paper)
- s_aff >> similarity obtained by affinity or Gaussian Kernel 

In [13]:
s_inn, s_aff = sg.generate_similarities(y=y_z)

In [14]:
ICESiss_results_ = SS.run_ANomalous_Cluster(pivot=seed, s=s_inn, p=p_u, rho=1, xi=1,)

### ICESins

In [15]:
ICESins_results_ = NS.run_ANomalous_Cluster(pivot=seed, s=s_inn, p=p_u, rho=1, xi=1,)

- The clustering results are dict of dict. 
To apply ARI from SKlearn, we need a 1-D array/list.
Thus we convert them by calling function flat_cluster_results.

In [16]:
ICESiss_results, _ = pt.flat_cluster_results(ICESiss_results_)

In [17]:
ICESins_results, _ = pt.flat_cluster_results(ICESins_results_)

In [18]:
print("ICESiss ARI:", metrics.adjusted_rand_score(gt, ICESiss_results), "\n"
      "ICESins ARI:", metrics.adjusted_rand_score(gt, ICESins_results),)

ICESiss ARI: 0.9130471309753844 
ICESins ARI: 0.6226053460736558


Note: Since the quality of recovered clustering results depends on the initialization point, thus in the paper, we reported the average and standard deviation of the results over 10 different initialization.
And here, we merely demonstrate a sample call for the proposed algorithms. 